## First runs (= runs where something is wrong)

In [ ]:
import os
import xarray as xr
import geopandas as gpd
import regionmask

In [ ]:
os.chdir('/home/rooda/Dropbox/Patagonia/')
period = slice("1980-01-01", "2019-12-31") # problem in CR2MET in 2019

In [ ]:
example    = xr.open_dataset("/home/rooda/Dropbox/Coding/OGGM/example_nc.nc")
dem        = xr.open_dataset("GIS South/dem_patagonia005.tif").rename({'y':'lat', 'x':'lon', 'band_data':'hgt'})
dem        = dem.sel(band=1, drop=True).drop("spatial_ref").fillna(0)
dem.hgt.attrs['units'] = "m"

rename_dic = {'latitude':'lat', 'longitude':'lon'}
pp_pmet    = xr.open_dataset("Data/Precipitation/PP_PMET_1980_2020m.nc", chunks = "auto").sel(time = period).rename(rename_dic).drop("crs")
t2m_pmet   = xr.open_dataset("Data/Temperature/Tavg_PMET_1980_2020m.nc", chunks = "auto").sel(time = period).rename(rename_dic)
t2m_pmet   = t2m_pmet.interp(lat = pp_pmet.lat, lon = pp_pmet.lon)
dem_pmet   = dem.interp(lat = pp_pmet.lat, lon = pp_pmet.lon).fillna(0)

pp_era5    = xr.open_dataset("Data/Precipitation/PP_ERA5_hr_1980_2020m.nc", chunks = "auto").sel(time = period).rename(rename_dic)
t2m_era5   = xr.open_dataset("Data/Temperature/Tavg_ERA5_hr_1980_2020m.nc", chunks = "auto").sel(time = period).rename(rename_dic)
dem_era5   = dem.interp(lat = pp_era5.lat, lon = pp_era5.lon).fillna(0)

pp_cr2met  = xr.open_dataset("Data/Precipitation/PP_CR2MET_1979_2020m.nc",  chunks = "auto").sel(time = period)
t2m_cr2met = xr.open_dataset("Data/Temperature/Tavg_CR2MET_1979_2020m.nc",  chunks = "auto").sel(time = period)
dem_cr2met = dem.interp(lat = pp_cr2met.lat, lon = pp_cr2met.lon).fillna(0)

pp_mswep  = xr.open_dataset("Data/Precipitation/PP_MSWEPv28_1979_2020m.nc", chunks = "auto").sel(time = period)
t2m_mswep = xr.open_dataset("Data/Temperature/Tavg_CR2MET_1979_2020m.nc",   chunks = "auto").sel(time = period)
t2m_mswep = t2m_mswep.interp(lat = pp_mswep.lat, lon = pp_mswep.lon)
dem_mswep = dem.interp(lat = pp_mswep.lat, lon = pp_mswep.lon).fillna(0)

In [ ]:
pmet   = xr.merge([pp_pmet,   t2m_pmet, dem_pmet]).rename({'pp':'prcp', 't2m':'temp'})

era5   = xr.merge([pp_era5,   t2m_era5, dem_era5]).rename({'pp':'prcp', 't2m':'temp'}).drop("crs")
era5   = era5.where((era5.lon >= -76) & (era5.lon <= -68) & (era5.lat >= -56) & (era5.lat <= -40), drop=True)
era5   = era5[["lon", "lat", "time", "prcp", "temp", "hgt"]]

cr2met = xr.merge([pp_cr2met, t2m_cr2met, dem_cr2met]).rename({'pp':'prcp', 't2m':'temp'})
cr2met = cr2met.where((cr2met.lon >= -76) & (cr2met.lon <= -68) & (cr2met.lat >= -56) & (cr2met.lat <= -40), drop=True)
cr2met = cr2met[["lon", "lat", "time", "prcp", "temp", "hgt"]]

mswep  = xr.merge([pp_mswep,  t2m_mswep, dem_mswep]).rename({'pp':'prcp', 't2m':'temp'})

In [ ]:
pmet.prcp.attrs['units']   = "mm"
era5.prcp.attrs['units']   = "mm"
cr2met.prcp.attrs['units'] = "mm"
mswep.prcp.attrs['units']  = "mm"

pmet.temp.attrs['units']   = "degC"
era5.temp.attrs['units']   = "degC"
cr2met.temp.attrs['units'] = "degC"
mswep.temp.attrs['units']  = "degC"

In [ ]:
# all basins in 9 selected zones 
basins = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_all.shp")[["Zone", "geometry"]]

# Glacier centroid location
ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6.shp")
ids = gpd.GeoDataFrame(ids, geometry=gpd.points_from_xy(ids.CenLon, ids.CenLat))

# Select the glaciers inside (centroid) the basin (TODO: move to glacier outlet)
ids = gpd.clip(ids, basins, keep_geom_type = False)
ids = ids.sjoin(basins, how="inner", predicate='intersects')
ids = ids.buffer(0.2) # mask to use for baseline climate

In [ ]:
mask_pmet     = regionmask.mask_geopandas(ids, pmet) >= 0
mask_era5     = regionmask.mask_geopandas(ids, era5) >= 0
mask_cr2met   = regionmask.mask_geopandas(ids, cr2met) >= 0
mask_mswep    = regionmask.mask_geopandas(ids, mswep) >= 0

In [ ]:
pmet    = pmet.where(mask_pmet, drop = True)
era5    = era5.where(mask_era5, drop = True)
cr2met  = cr2met.where(mask_cr2met, drop = True)
mswep   = mswep.where(mask_mswep, drop = True)

In [ ]:
pmet.to_netcdf("/home/rooda/OGGM_results/PMET_OGGM_1980_2019m.nc")
era5.to_netcdf("/home/rooda/OGGM_results/ERA5_OGGM_1980_2019m.nc")
cr2met.to_netcdf("/home/rooda/OGGM_results/CR2MET_OGGM_1980_2019m.nc") 
mswep.to_netcdf("/home/rooda/OGGM_results/MSWEP_OGGM_1980_2019m.nc")